# API RED ELECTRICA DE ESPAÑA - GENERACIÓN Y EMISIONES

Vamos a obtener datos de la web de Red Eléctrica de España (https://www.ree.es/es) usando su API y siguiendo la información de la siguiente documentación (https://www.ree.es/es/apidatos).

Los datos a manejar en el presente Notebook serán los relativos a la generación energética a partir de las diferentes tecnologías disponibles, tanto renovables como no renovables y las emisiones de CO2 resultantes.
    

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

import json
from pandas import Timestamp

import requests
from pprint import pprint

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# EXTRACCIÓN

## Automatización de la extracción

Para la categoría de generación de la API se decide generar dos funciones relacionadas con la generación de energía por tipo de tecnología (la primera) y otra para la emisión total de CO2 de las diferentes energías no renovables (la segunda). De ambas funciones se extraen valores anuales para una mejor y más clara interpretación de los datos.

In [2]:
def generacion_x_tecnologia(inicio, fin):
    
    if inicio < 2014:
        
        return print("Para obtener información completa de todas las tecnologías disponibles, la fecha de inicio de búsqueda debe ser posterior a 2014")
    
    else:
        
        search_url = "https://apidatos.ree.es/es/datos/"

        time_trunc="year"  

        category="generacion"

        widgets="estructura-generacion-emisiones-asociadas"

        Data_gen={}

        for x in range(inicio,fin):
            start_date = f"{x}-01-01T00:00"
            end_date= f"{x}-12-31T23:59"
            query = f"start_date={start_date}&end_date={end_date}&time_trunc={time_trunc}"
            endpoint = f"{search_url}{category}/{widgets}?{query}"
            response = requests.get(url = endpoint)
            data = response.json()
            Data_gen[f"{x}"]=data

        años=[]
        gen_hidraulica=[]
        gen_nuclear=[]
        gen_carbon=[]
        gen_mdiesel=[]
        gen_tugas=[]
        gen_tuvap=[]
        gen_cicombo=[]
        gen_hidroeolica=[]
        gen_eolica=[]
        gen_solfot=[]
        gen_solterm=[]
        gen_otren=[]
        gen_cogen=[]
        gen_resnoren=[]
        gen_resren=[]

        for x in Data_gen.keys():

            años.append(x)

            for i in Data_gen[x]["included"]:

                if i["type"] =="Hidráulica":

                    gen_hidraulica.append(i["attributes"]["values"][0]["value"])

                if i["type"] =="Nuclear":

                    gen_nuclear.append(i["attributes"]["values"][0]["value"]) 

                if i["type"] =="Carbón":

                    gen_carbon.append(i["attributes"]["values"][0]["value"])         

                if i["type"] =="Motores diésel":

                    gen_mdiesel.append(i["attributes"]["values"][0]["value"])    

                if i["type"] =="Turbina de gas":

                    gen_tugas.append(i["attributes"]["values"][0]["value"])    

                if i["type"] =="Turbina de vapor":

                    gen_tuvap.append(i["attributes"]["values"][0]["value"]) 

                if i["type"] =="Ciclo combinado":

                    gen_cicombo.append(i["attributes"]["values"][0]["value"]) 

                if i["type"] =="Hidroeólica":

                    gen_hidroeolica.append(i["attributes"]["values"][0]["value"])   

                if i["type"] =="Eólica":

                    gen_eolica.append(i["attributes"]["values"][0]["value"])            

                if i["type"] =="Solar fotovoltaica":

                    gen_solfot.append(i["attributes"]["values"][0]["value"])  

                if i["type"] =="Solar térmica":

                    gen_solterm.append(i["attributes"]["values"][0]["value"])  

                if i["type"] =="Otras renovables":

                    gen_otren.append(i["attributes"]["values"][0]["value"]) 

                if i["type"] =="Cogeneración":

                    gen_cogen.append(i["attributes"]["values"][0]["value"])

                if i["type"] =="Residuos no renovables":

                    gen_resnoren.append(i["attributes"]["values"][0]["value"])

                if i["type"] =="Residuos renovables":

                    gen_resren.append(i["attributes"]["values"][0]["value"])

        df=pd.DataFrame({"Años": años,
                         "Generación Hidráulica MWh":gen_hidraulica,
                         "Generación Nuclear MWh":gen_nuclear,
                         "Generación Carbón MWh":gen_carbon,
                         "Generación Motores diésel MWh":gen_mdiesel,
                         "Generación Turbina de gas MWh":gen_tugas,
                         "Generación Turbina de vapor MWh":gen_tuvap,
                         "Generación Ciclo combinado MWh":gen_cicombo,
                         "Generación Hidroeólica MWh":gen_hidroeolica,
                         "Generación Eólica MWh":gen_eolica,
                         "Generación Solar fotovoltaica MWh":gen_solfot,
                         "Generación Solar térmica MWh":gen_solterm,
                         "Generación Otras renovables MWh":gen_otren,
                         "Generación Cogeneración MWh":gen_cogen,
                         "Generación Residuos no renovables MWh":gen_resnoren,
                         "Generación Residuos renovables MWh":gen_resren})
        return df
    

In [4]:
gen_x_tecnologia=generacion_x_tecnologia(2014 , 2023)

In [5]:
gen_x_tecnologia

,Años,Generación Hidráulica MWh,Generación Nuclear MWh,Generación Carbón MWh,Generación Motores diésel MWh,Generación Turbina de gas MWh,Generación Turbina de vapor MWh,Generación Ciclo combinado MWh,Generación Hidroeólica MWh,Generación Eólica MWh,Generación Solar fotovoltaica MWh,Generación Solar térmica MWh,Generación Otras renovables MWh,Generación Cogeneración MWh,Generación Residuos no renovables MWh,Generación Residuos renovables MWh
0,2014,3.918199e+07,5.478128e+07,4.324606e+07,3227383.088,946991.236,2070771.428,2.482883e+07,889.277,5.103203e+07,8.207926e+06,4958914.926,3816315.717,2.415324e+07,1.965877e+06,678071.5725
1,2015,2.838258e+07,5.466180e+07,5.261648e+07,3345074.394,915771.849,2222950.567,2.902729e+07,8207.424,4.811789e+07,8.243561e+06,5085235.514,3432591.921,2.520088e+07,2.480109e+06,818049.7025
2,2016,3.611489e+07,5.602168e+07,3.731378e+07,3602463.490,616037.302,2536143.003,2.900648e+07,17891.936,4.769666e+07,7.977467e+06,5071201.702,3425664.828,2.590864e+07,2.606964e+06,785395.0295
3,2017,1.845062e+07,5.553935e+07,4.501942e+07,3456023.552,871160.048,2674393.850,3.706579e+07,20233.057,4.790696e+07,8.397753e+06,5347952.465,3610347.882,2.821181e+07,2.607985e+06,877006.0870
4,2018,3.411724e+07,5.319762e+07,3.727681e+07,3178179.141,1049279.011,2455432.297,3.004447e+07,23655.544,4.958149e+07,7.766178e+06,4424326.674,3557439.110,2.900676e+07,2.434963e+06,874075.2450
5,2019,2.471902e+07,5.582423e+07,1.267064e+07,2836053.144,670532.077,2189010.668,5.524197e+07,23248.718,5.424506e+07,9.252019e+06,5166431.145,3617715.229,2.961514e+07,2.222463e+06,889813.7500
6,2020,3.063172e+07,5.575836e+07,5.020713e+06,2399492.159,406582.811,1387607.524,4.402286e+07,19540.227,5.490625e+07,1.530214e+07,4538310.130,4481581.196,2.703031e+07,2.016188e+06,725652.2040
7,2021,2.962601e+07,5.404102e+07,4.983148e+06,2517424.107,423579.904,1108055.957,4.450010e+07,23098.239,6.052566e+07,2.098104e+07,4705524.869,4719873.222,2.609065e+07,2.238741e+06,877940.7310
8,2022,1.790701e+07,5.598361e+07,7.765054e+06,2548201.294,657477.274,1207233.064,6.813750e+07,22880.844,6.119357e+07,2.790197e+07,4123192.828,4656917.299,1.775422e+07,1.899961e+06,877707.6180


In [6]:
def emisionesCO2_no_renovables(inicio, fin):
    
    #inicio superior a 2014 debido a los datos disponibles
    
    if inicio < 2007:
        
        return print("No se dispone de valores anteriores al 2007.")
    
    elif fin > 2023:
        
        return print("No se dispone de valores promedio anuales para años no transcurridos.")        
   
    else:
        
        search_url = "https://apidatos.ree.es/es/datos/"

        time_trunc="year"  

        category="generacion"

        widgets="no-renovables-detalle-emisiones-CO2"

        Data_gen={}

        for x in range(inicio,fin):
            start_date = f"{x}-01-01T00:00"
            end_date= f"{x}-12-31T23:59"
            query = f"start_date={start_date}&end_date={end_date}&time_trunc={time_trunc}"
            endpoint = f"{search_url}{category}/{widgets}?{query}"
            response = requests.get(url = endpoint)
            data = response.json()
            Data_gen[f"{x}"]=data

        años=[]
        em_carbon=[]
        em_mdiesel=[]
        em_tugas=[]
        em_tuvap=[]
        em_cicombo=[]
        em_cogen=[]
        em_resnoren=[]
        em_total=[]
        em_totalMWh=[]
        gen_x_em=[]

        for x in Data_gen.keys():

            años.append(x)

            for i in Data_gen[x]["included"]:

                if i["type"] =="Carbón":

                    em_carbon.append(i["attributes"]["values"][0]["value"])        

                if i["type"] =="Motores diésel":

                    em_mdiesel.append(i["attributes"]["values"][0]["value"])    

                if i["type"] =="Turbina de gas":

                    em_tugas.append(i["attributes"]["values"][0]["value"])    

                if i["type"] =="Turbina de vapor":

                    em_tuvap.append(i["attributes"]["values"][0]["value"]) 

                if i["type"] =="Ciclo combinado":

                    em_cicombo.append(i["attributes"]["values"][0]["value"]) 

                if i["type"] =="Cogeneración":

                    em_cogen.append(i["attributes"]["values"][0]["value"])

                if i["type"] =="Residuos no renovables":

                    em_resnoren.append(i["attributes"]["values"][0]["value"])

                if i["type"] =="Total tCO2 eq.":

                    em_total.append(i["attributes"]["values"][0]["value"])

                if i["type"] =="tCO2 eq./MWh":

                    em_totalMWh.append(i["attributes"]["values"][0]["value"])
            
            
        df=pd.DataFrame({"Años": años,
                         "Emisiones Carbón":em_carbon,
                         "Emisiones Motores diésel":em_mdiesel,
                         "Emisiones Turbina de gas":em_tugas,
                         "Emisiones Turbina de vapor":em_tuvap,
                         "Emisiones Ciclo combinado":em_cicombo,
                         "Emisiones Cogeneración":em_cogen,
                         "Emisiones Residuos no renovables":em_resnoren,
                         "Total tCO2 eq.":em_total,
                         "tCO2 eq./MWh":em_totalMWh})
        return df


In [7]:
emisiones=emisionesCO2_no_renovables(2014,2023)

In [8]:
emisiones

,Años,Emisiones Carbón,Emisiones Motores diésel,Emisiones Turbina de gas,Emisiones Turbina de vapor,Emisiones Ciclo combinado,Emisiones Cogeneración,Emisiones Residuos no renovables,Total tCO2 eq.,tCO2 eq./MWh
0,2014,4.109070e+07,2599548.959,756443.488,1842986.573,1.053624e+07,9.178233e+06,471810.509,6.647597e+07,0.252668
1,2015,4.998565e+07,2651845.256,726326.613,1978426.001,1.204763e+07,9.576334e+06,595226.166,7.756145e+07,0.293173
2,2016,3.544809e+07,2811794.946,532295.390,2257167.264,1.196615e+07,9.845285e+06,625671.295,6.348645e+07,0.245404
3,2017,4.276845e+07,2728450.366,692788.968,2380210.527,1.494380e+07,1.072049e+07,625916.275,7.486010e+07,0.287861
4,2018,3.600179e+07,2234468.118,1045096.297,2209889.087,1.184192e+07,1.102257e+07,584391.068,6.494012e+07,0.250746
5,2019,1.238427e+07,1998445.302,675945.145,1970362.334,2.118392e+07,1.125375e+07,533391.187,5.000009e+07,0.192914
6,2020,4.884685e+06,1641040.380,430930.401,1259998.633,1.713342e+07,1.006875e+07,712029.329,3.613085e+07,0.145310
7,2021,4.865740e+06,1720961.517,451513.465,1007827.637,1.739329e+07,9.688053e+06,780958.864,3.590834e+07,0.139525
8,2022,7.489053e+06,1742437.026,703604.641,1094299.003,2.619600e+07,6.645748e+06,575052.489,4.444620e+07,0.163024


# Transformación

Se pasa la energía generada de MWh a GWh

In [9]:
for x in gen_x_tecnologia.columns:
    
    if x != "Años":
        
        gen_x_tecnologia[x]=gen_x_tecnologia[x].apply(lambda i : round(i/1000,2))

Se redondean los valores de emisiones para que salgan únicamente con 2 decimales

In [10]:
for x in emisiones.columns:
    
    if x != "Años":
        
        emisiones[x]=emisiones[x].apply(lambda i : round(i,2))

Como se cambiaron las unidades, se renombran las columnas también.

In [11]:
gen_x_tecnologia.rename(columns={"Generación Hidráulica MWh":"Generación Hidráulica GWh",
                                 "Generación Nuclear MWh":"Generación Nuclear GWh",
                                 "Generación Carbón MWh":"Generación Carbón GWh",
                                 "Generación Motores diésel MWh":"Generación Motores diésel GWh",
                                 "Generación Turbina de gas MWh":"Generación Turbina de gas GWh",
                                 "Generación Turbina de vapor MWh":"Generación Turbina de vapor GWh",
                                 "Generación Ciclo combinado MWh":"Generación Ciclo combinado GWh",
                                 "Generación Hidroeólica MWh":"Generación Hidroeólica GWh",
                                 "Generación Eólica MWh":"Generación Eólica GWh",
                                 "Generación Solar fotovoltaica MWh":"Generación Solar fotovoltaica GWh",
                                 "Generación Solar térmica MWh":"Generación Solar térmica GWh",
                                 "Generación Otras renovables MWh":"Generación Otras renovables GWh",
                                 "Generación Cogeneración MWh":"Generación Cogeneración GWh",
                                 "Generación Residuos no renovables MWh":"Generación Residuos no renovables GWh",
                                 "Generación Residuos renovables MWh":"Generación Residuos renovables GWh"}, inplace=True)

In [12]:
gen_x_tecnologia

,Años,Generación Hidráulica GWh,Generación Nuclear GWh,Generación Carbón GWh,Generación Motores diésel GWh,Generación Turbina de gas GWh,Generación Turbina de vapor GWh,Generación Ciclo combinado GWh,Generación Hidroeólica GWh,Generación Eólica GWh,Generación Solar fotovoltaica GWh,Generación Solar térmica GWh,Generación Otras renovables GWh,Generación Cogeneración GWh,Generación Residuos no renovables GWh,Generación Residuos renovables GWh
0,2014,39181.99,54781.28,43246.06,3227.38,946.99,2070.77,24828.83,0.89,51032.03,8207.93,4958.91,3816.32,24153.24,1965.88,678.07
1,2015,28382.58,54661.80,52616.48,3345.07,915.77,2222.95,29027.29,8.21,48117.89,8243.56,5085.24,3432.59,25200.88,2480.11,818.05
2,2016,36114.89,56021.68,37313.78,3602.46,616.04,2536.14,29006.48,17.89,47696.66,7977.47,5071.20,3425.66,25908.64,2606.96,785.40
3,2017,18450.62,55539.35,45019.42,3456.02,871.16,2674.39,37065.79,20.23,47906.96,8397.75,5347.95,3610.35,28211.81,2607.98,877.01
4,2018,34117.24,53197.62,37276.81,3178.18,1049.28,2455.43,30044.47,23.66,49581.49,7766.18,4424.33,3557.44,29006.76,2434.96,874.08
5,2019,24719.02,55824.23,12670.64,2836.05,670.53,2189.01,55241.97,23.25,54245.06,9252.02,5166.43,3617.72,29615.14,2222.46,889.81
6,2020,30631.72,55758.36,5020.71,2399.49,406.58,1387.61,44022.86,19.54,54906.25,15302.14,4538.31,4481.58,27030.31,2016.19,725.65
7,2021,29626.01,54041.02,4983.15,2517.42,423.58,1108.06,44500.10,23.10,60525.66,20981.04,4705.52,4719.87,26090.65,2238.74,877.94
8,2022,17907.01,55983.61,7765.05,2548.20,657.48,1207.23,68137.50,22.88,61193.57,27901.97,4123.19,4656.92,17754.22,1899.96,877.71


Se juntan los 2 data frames para la energía generada y las emisiones de CO2 por tecnología.

In [13]:
df_merged = pd.merge(left = gen_x_tecnologia, right = emisiones, left_on="Años", right_on="Años")


In [14]:
df_merged

,Años,Generación Hidráulica GWh,Generación Nuclear GWh,Generación Carbón GWh,Generación Motores diésel GWh,Generación Turbina de gas GWh,Generación Turbina de vapor GWh,Generación Ciclo combinado GWh,Generación Hidroeólica GWh,Generación Eólica GWh,...,Generación Residuos renovables GWh,Emisiones Carbón,Emisiones Motores diésel,Emisiones Turbina de gas,Emisiones Turbina de vapor,Emisiones Ciclo combinado,Emisiones Cogeneración,Emisiones Residuos no renovables,Total tCO2 eq.,tCO2 eq./MWh
0,2014,39181.99,54781.28,43246.06,3227.38,946.99,2070.77,24828.83,0.89,51032.03,...,678.07,41090701.18,2599548.96,756443.49,1842986.57,10536239.85,9178232.70,471810.51,66475974.29,0.25
1,2015,28382.58,54661.80,52616.48,3345.07,915.77,2222.95,29027.29,8.21,48117.89,...,818.05,49985653.75,2651845.26,726326.61,1978426.00,12047630.32,9576333.59,595226.17,77561453.33,0.29
2,2016,36114.89,56021.68,37313.78,3602.46,616.04,2536.14,29006.48,17.89,47696.66,...,785.40,35448088.89,2811794.95,532295.39,2257167.26,11966149.07,9845284.57,625671.30,63486451.59,0.25
3,2017,18450.62,55539.35,45019.42,3456.02,871.16,2674.39,37065.79,20.23,47906.96,...,877.01,42768449.39,2728450.37,692788.97,2380210.53,14943795.13,10720486.58,625916.27,74860097.24,0.29
4,2018,34117.24,53197.62,37276.81,3178.18,1049.28,2455.43,30044.47,23.66,49581.49,...,874.08,36001786.05,2234468.12,1045096.30,2209889.09,11841921.44,11022567.76,584391.07,64940119.82,0.25
5,2019,24719.02,55824.23,12670.64,2836.05,670.53,2189.01,55241.97,23.25,54245.06,...,889.81,12384273.16,1998445.30,675945.15,1970362.33,21183920.12,11253752.86,533391.19,50000090.11,0.19
6,2020,30631.72,55758.36,5020.71,2399.49,406.58,1387.61,44022.86,19.54,54906.25,...,725.65,4884685.30,1641040.38,430930.40,1259998.63,17133424.04,10068746.46,712029.33,36130854.55,0.15
7,2021,29626.01,54041.02,4983.15,2517.42,423.58,1108.06,44500.10,23.10,60525.66,...,877.94,4865739.92,1720961.52,451513.47,1007827.64,17393286.98,9688052.82,780958.86,35908341.21,0.14
8,2022,17907.01,55983.61,7765.05,2548.20,657.48,1207.23,68137.50,22.88,61193.57,...,877.71,7489053.35,1742437.03,703604.64,1094299.00,26196004.51,6645748.13,575052.49,44446199.15,0.16


# VISUALIZACIONES

**GENERACIÓN ENERGÉTICA POR TECNOLOGÍA Y AÑO**

In [23]:
fig=px.bar(data_frame = gen_x_tecnologia,
       x          = "Años",
       y          = gen_x_tecnologia.columns[1:],
       title      ="Generación energética por tipo de tecnología utilizada")

fig.show()

**EVOLUCIÓN DE LA GENERACIÓN ENERGÉTICA POR TECNOLOGÍA**

In [24]:
px.line(data_frame = gen_x_tecnologia,
        x = "Años",
        y = gen_x_tecnologia.columns[1:],
        title = "Evolución de la generación energética por tecnología")

**EMISIONES DE CO2 POR TECNOLOGÍA NO RENOVABLE Y AÑO**

In [25]:
px.bar(data_frame = emisiones,
       x          = "Años",
       y          = emisiones.columns[1:-2],
       title = "Emisiones de CO2")


**RELACIÓN ENTRE LA ENERGÍA GENERADA Y LAS EMISIONES DE CO2 EQUIVALENTES**

In [26]:
px.line(data_frame = emisiones,
        x = "Años",
        y = "tCO2 eq./MWh",
        title = "Relación entre emisiones de CO2 y energía generada")